# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_ind_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [106]:
#!pip install xlrd
#!pip install openpyxl
import pandas as pd
import numpy as np
import re

digits_regex = r'\d+'
parenthesis_regex = r' +\(.+\)'


limit_rank = 15

last_n_years = [str(num) for num in list(range(2006, 2016, 1))]

    
countries_to_rename_energy = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}

countries_to_rename_gpd = {
    "Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}

tree_dots_to_nan = lambda x: np.nan if x == "..." else x

def format_countries_with_num_or_parenthesis(country: str):
    
    to_return = country
    
    if(re.search(digits_regex, country)):
        to_return = re.sub(digits_regex, '', country)
    elif(re.search(parenthesis_regex, country)):
        to_return = re.sub(parenthesis_regex, '', country)
        
    return to_return

def get_generate_energy_ind_df():

    energy_ind_df = pd.read_excel('assets/Energy Indicators.xls', header=None)
    energy_ind_df = energy_ind_df[18:245].reset_index()
    energy_ind_df = energy_ind_df.drop(columns=['index', 0, 1], axis=1)
    energy_ind_df = energy_ind_df.rename(columns={2: "Country", 3: "Energy Supply", 4: "Energy Supply per Capita", 5: "% Renewable"})
      
    energy_ind_df[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = \
    energy_ind_df[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].map(tree_dots_to_nan)
    energy_ind_df['Energy Supply'] = energy_ind_df['Energy Supply'].apply(lambda x: x*1000000)
    
    # YOUR CODE HERE transform Country column
    energy_ind_df['Country'] = energy_ind_df['Country'].apply(lambda c: format_countries_with_num_or_parenthesis(c))
    energy_ind_df['Country'] = energy_ind_df['Country'].replace(countries_to_rename_energy)
    
    return energy_ind_df

def get_GDP_df():

    # YOUR CODE HERE load data, remove header, adjust column names
    gdp_df = pd.read_csv('assets/world_bank.csv', skiprows=4)
    gdp_df = gdp_df.rename(columns={'Country Name': 'Country'})
    cols = ['Country', *last_n_years]
    gdp_df = gdp_df[cols]
    
    gdp_df['Country'] = gdp_df['Country'].replace(countries_to_rename_gpd)
    
    return gdp_df

def get_scimago_df():
    
    scimago_df = pd.read_excel('assets/scimagojr-3.xlsx')
    
    return scimago_df

def answer_one():
    
    generate_energy_ind_df = get_generate_energy_ind_df()
    gpd_df = get_GDP_df()
    scimago_df = get_scimago_df()
    
    merged_df = generate_energy_ind_df.merge(scimago_df.merge(gpd_df, on='Country'), on='Country')
    merged_df = merged_df.set_index('Country')
    merged_df = merged_df.sort_values(by='Rank', ascending=True)[:limit_rank:]
    
    return merged_df
answer_one()

,Energy Supply,Energy Supply per Capita,% Renewable,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1.271910e+11,93.0,19.754910,1,127050,126767,597237,411683,4.70,138,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,9.083800e+10,286.0,11.570980,2,96661,94747,792274,265436,8.20,230,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,1.898400e+10,149.0,10.232820,3,30504,30287,223024,61554,7.31,134,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,7.920000e+09,124.0,10.600470,4,20944,20357,206091,37874,9.84,139,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,3.070900e+10,214.0,17.288680,5,18534,18301,34266,12422,1.85,57,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,1.043100e+10,296.0,61.945430,6,17899,17620,215003,40930,12.01,149,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,1.326100e+10,165.0,17.901530,7,17027,16831,140566,27426,8.26,126,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,3.319500e+10,26.0,14.969080,8,15005,14841,128763,37209,8.58,115,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,1.059700e+10,166.0,17.020280,9,13153,12973,130632,28601,9.93,114,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [107]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

In [108]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [109]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [110]:
def answer_two():
    generate_energy_ind_df = get_generate_energy_ind_df()
    gpd_df = get_GDP_df()
    scimago_df = get_scimago_df()
    
    union_df = generate_energy_ind_df.merge(scimago_df.merge(gpd_df, on='Country'), on='Country', how='outer')
    intersect_df = generate_energy_ind_df.merge(scimago_df.merge(gpd_df, on='Country'), on='Country')
    
    lost_entries = union_df.shape[0] - intersect_df.shape[0]
    
    return lost_entries
    
answer_two()

70

In [111]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

In [112]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [113]:
def answer_three():
    answer_one_df = answer_one()
    
    avg_gdp = answer_one_df[last_n_years].mean(axis=1)
    avg_gdp = avg_gdp.sort_values(ascending=False)
    
    return avg_gdp

answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [114]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

In [115]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [116]:
def answer_four():
    answer_one_df = answer_one()
    
    largest_avertage_gdp_countries = answer_three().index
    
    six_th_country = largest_avertage_gdp_countries[5]

    initial_gdp = answer_one_df.loc[six_th_country][last_n_years[0]]
    final_gdp = answer_one_df.loc[six_th_country][last_n_years[-1]]
    
    how_much_changed = float(initial_gdp - final_gdp)
    
    return how_much_changed

In [117]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

In [118]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [119]:
def answer_five():
    
    answer_one_df = answer_one()
    energy_supply_per_capita = answer_one_df['Energy Supply per Capita']
    mean_espc = float(energy_supply_per_capita.mean())
    
    return mean_espc

answer_five()

157.6

In [120]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

In [121]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [122]:
def answer_six():
    
    answer_one_df = answer_one()
    
    percentage_renewable = answer_one_df['% Renewable']
    
    percentage_renewable_max_name = percentage_renewable.idxmax()
    percentage_renewable_max_value = float(percentage_renewable.max())
    
    return (percentage_renewable_max_name, percentage_renewable_max_value)

answer_six()

('Brazil', 69.64803)

In [123]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

In [124]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [125]:
def answer_seven():
    
    answer_one_df = answer_one()
    
    answer_one_df['Ratio Citations'] = answer_one_df['Self-citations'] / answer_one_df['Citations']
    
    ratio_citations_max_name = answer_one_df['Ratio Citations'].idxmax()
    ratio_citations_max_value = float(answer_one_df['Ratio Citations'].max())
    
    return (ratio_citations_max_name, ratio_citations_max_value)

answer_seven()

('China', 0.6893126179389422)

In [126]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

In [127]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [128]:
def answer_eight():
    
    answer_one_df = answer_one()
    
    # feature engineering
    answer_one_df['Population Estimate'] = answer_one_df['Energy Supply'] / answer_one_df['Energy Supply per Capita']
    
    # index property gives me an array with loc index sorted previously
    sorted_countries = answer_one_df['Population Estimate'].sort_values(ascending=False).index
    
    # I put 2 to get 3rd element 'cause the list index starts in 0
    third_populous_country_name = sorted_countries[2]

    return third_populous_country_name

answer_eight()

'United States'

In [129]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

In [130]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [131]:
def answer_nine():
    
    answer_one_df = answer_one()
    
    # feature engineering
    answer_one_df['Population Estimate'] = answer_one_df['Energy Supply'] / answer_one_df['Energy Supply per Capita'] 
    answer_one_df['Citable Documents per Capita'] = answer_one_df['Citable documents'] / answer_one_df['Population Estimate']
    
    
    corr = float(answer_one_df['Citable Documents per Capita'].corr(answer_one_df['Energy Supply per Capita'], method='pearson'))
    
    return corr

answer_nine()

0.7940010435442946

In [132]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

In [133]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [134]:
def answer_ten():
    
    answer_one_df = answer_one()
    
    percentage_renewable = answer_one_df['% Renewable']
    percentage_renewable_median = percentage_renewable.median()
    
    # Here i used a ternary operator to simplify validation
    answer_one_df['Renewable Classification'] = percentage_renewable.apply(lambda p: 1 if p >= percentage_renewable_median else 0)
    
    classification = answer_one_df['Renewable Classification']
    
    return classification

answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: Renewable Classification, dtype: int64

In [135]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

In [136]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [137]:
def answer_eleven():
    ContinentDict = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }
            
    answer_one_df = answer_one()
    
    answer_one_df['Population Estimate'] = answer_one_df['Energy Supply'] / answer_one_df['Energy Supply per Capita']
    population_estimate = answer_one_df['Population Estimate'].reset_index()
    population_estimate['Continent'] = population_estimate['Country'].map(ContinentDict)
    
    grouped_df = population_estimate.groupby('Continent')['Population Estimate'].agg(['size', 'sum', 'mean', 'std'])
    
    return grouped_df

answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [138]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

In [139]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [140]:
def answer_twelve():
    
    ContinentDict = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }
    
    answer_one_df = answer_one()
            
    renewable_bins = pd.cut(answer_one_df['% Renewable'], 5)
    
    # I don't know if is legal use observed=True to avoid groups with no countries but ir works xd
    grouped = answer_one_df.groupby([ContinentDict, renewable_bins], observed=True).size()
        
    return grouped

answer_twelve()

Country        % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

In [141]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"

In [142]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [143]:
def answer_thirteen():
    
    answer_one_df = answer_one()
    
    # Calculate the population estimate of each country
    answer_one_df['Estimated Population'] = answer_one_df['Energy Supply'] / answer_one_df['Energy Supply per Capita']
    
    # Convert the population estimate to a string with thousands separator
    pop_est = answer_one_df['Estimated Population'].apply(lambda x: "{:,.0f}".format(x).replace(",", "."))
    
    return pop_est

df_result = answer_thirteen()
df_result

Country
China                 1.367.645.161
United States           317.615.385
Japan                   127.409.396
United Kingdom           63.870.968
Russian Federation      143.500.000
Canada                   35.239.865
Germany                  80.369.697
India                 1.276.730.769
France                   63.837.349
South Korea              49.805.430
Italy                    59.908.257
Spain                    46.443.396
Iran                     77.075.630
Australia                23.316.017
Brazil                  205.915.254
Name: Estimated Population, dtype: object

In [144]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

In [145]:
# space for proffesor tests